In [11]:
import pandas as pd
# read data
length_acce = pd.read_table('/home/ouconstand/Learning/graduation/virus_host/construct_graph/Virion_accession_length.txt', sep='\t')
acce_id = pd.read_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/final_acce_virusID.csv')

length_acce.columns = ['Accession','Length']
# move duplication
length_acce.drop_duplicates(['Accession','Length'],inplace=True)

In [31]:
old_data = set(length_acce['Accession'])
new_data = set(acce_id['Accession'])
print(len(old_data))
print(len(new_data))

1023868
1025047


In [33]:
different_data = list((old_data | new_data)-old_data)
len(different_data)

1290

In [102]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

# the function is designed to get fasta format information
api_key = '59cbd6ff6b3ce5d506f0eeb69e4e09362409'
def get_url_info(segment):

    # Using requests function get the id from term
    term_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&api_key={}&term='.format(api_key) + ",".join(segment)
    term_info = BeautifulSoup(requests.get(term_url, timeout=(10,30)).text, 'html.parser')

    # get id
    pattern = re.compile(r'<id>(.*)</id>')
    accession_id = re.findall(pattern, str(term_info))
    
    # get sequencing information from id
    fasta_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&id={}&api_key={}'.format(','.join(accession_id), api_key)
    fasta_info = requests.get(fasta_url, timeout=(10,30)).text
    return fasta_info

# this function is designed to write files
def write_file(fasta_info,name_num):
    with open("/home/ouconstand/data/Virus_Host/sequence_{}.fasta".format(name_num), 'w') as f:
        f.write(fasta_info)

In [103]:
import math
for num in range(math.ceil(len(different_data) / 20)):
    write_file(get_url_info(different_data[num*20:(num+1)*20]), num)